**I. Подключение к БД**

*Импортируем библиотеку psycopg2*

In [ ]:
import psycopg2

*Создаём подключение к БД*

In [ ]:
conn = psycopg2.connect(host="rc1b-7ng6ih3jte3824x8.mdb.yandexcloud.net", 
                        port=6432,
                        database="demo", 
                        user="student",
                        password="student!"
                       )

*Взаимодействие с базой осуществляется при помощи отдельного класса, cursor*

In [ ]:
cur = conn.cursor()

In [ ]:
sql = '''
SELECT *
  FROM bookings.aircrafts_data
'''

In [ ]:
print(sql)

*Забрать все данные*

In [ ]:
cur.execute(sql)
result = cur.fetchall()

In [ ]:
result

*Забрать данные по очереди*

In [ ]:
cur.execute(sql)

In [ ]:
cur.fetchone()

*Вывод данных в цикле*

In [ ]:
cur.execute(sql)
for row in cur:
    print(row)

*По завершении работы с курсором его следует закрыть*

In [ ]:
cur.close()

*Чтобы постоянно не думать про cur.close(), можно выполнять транзакции в with-блоках:*

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute(sql)
        result = cur.fetchall()

In [ ]:
result

**II. Подключение к БД через SQLAlchemy**

Особенности SQLAlchemy:

- ORM или Object Relational Mapper (объектно-реляционное отображение) позволяет работать с базой данных с помощью объектно-ориентированного кода, не используя SQL-запросы.
- Код будет оставаться одинаковым вне зависимости от используемой БД.

In [ ]:
# !pip install sqlalchemy

In [ ]:
import sqlalchemy

sqlalchemy.__version__

По умолчанию SQLAlchemy работает только с базой данных SQLite без дополнительных драйверов. 

Для работы с другими базами данных необходимо установить DBAPI-совместимый драйвер в соответствии с базой данных.

Далее перечислены основные драйверы для различных БД:

- MySQL: PyMySQL, MySQL-Connector, CyMySQL, MySQL-Python 
- PostgreSQL: psycopg2, pg8000
- Microsoft SQL Server: PyODBC, pymssql
- Oracle: cx-Oracle
- Firebird: fdb, kinterbasdb

In [ ]:
from sqlalchemy import create_engine

In [ ]:
def connection_yandex_cloud_demo(echo):
    """Connection to DataBase demo"""
    login = 'student'
    password = 'student!'
    host = 'rc1b-7ng6ih3jte3824x8.mdb.yandexcloud.net'
    port = '6432'
    db_name = 'demo'
    return create_engine(f'postgresql://{login}:{password}@{host}:{port}/{db_name}', echo=echo)

In [ ]:
engine = connection_yandex_cloud_demo(echo=True)

Формирование запроса на выборку данных

In [ ]:
engine.table_names()

*Запрос на чистом SQL*

In [ ]:
select_result = engine.execute('''select * from aircrafts_data''').fetchall()

In [ ]:
select_result

*Результат возвращается в виде списка. Каждый элемент списка является строкой из БД.*

In [ ]:
select_result[0]

In [ ]:
select_result[0][0]

In [ ]:
select_result[0][1]

In [ ]:
select_result[0][1]

In [ ]:
select_result[0][1]['en']

*Запросы с применением ORM (объектно-реляционное отображение) SQLAlchemy*

Загружаем данные о таблицах из БД demo

In [ ]:
from sqlalchemy.schema import MetaData

metadata_demo_db = MetaData()
metadata_demo_db.reflect(bind=engine);

In [ ]:
for table_name in metadata_demo_db.tables:
    print(table_name)

In [ ]:
from sqlalchemy import Table

boarding_passes = Table('boarding_passes', metadata_demo_db, autoload=True)
ticket_flights = Table('ticket_flights', metadata_demo_db, autoload=True)
flights = Table('flights', metadata_demo_db, autoload=True)
aircrafts_data = Table('aircrafts_data', metadata_demo_db, autoload=True)
airports_data = Table('airports_data', metadata_demo_db, autoload=True)
tickets = Table('tickets', metadata_demo_db, autoload=True)
bookings = Table('bookings', metadata_demo_db, autoload=True)
seats = Table('seats', metadata_demo_db, autoload=True)

In [ ]:
print(aircrafts_data.columns)

In [ ]:
from sqlalchemy import select

sql_query = select(aircrafts_data)
result = engine.execute(sql_query)

In [ ]:
result.fetchall()

Фильтрация записей

In [ ]:
sql_query = select(aircrafts_data).where(
    aircrafts_data.c.range < 5000
)


print(sql_query)

In [ ]:
result_with_where = engine.execute(sql_query);

result_with_where.fetchall()

Фильтрация по нескольким условиям

In [ ]:
sql_query = select(aircrafts_data).where(
    (aircrafts_data.c.range < 5000) &
    (aircrafts_data.c.aircraft_code == 'SU9')
)

print(sql_query)

In [ ]:
result_with_where = engine.execute(sql_query);

result_with_where.fetchall()

Логические операторы:

AND - "&"

OR - "|"

NOT - "~" 

In [ ]:
sql_query = select([aircrafts_data]).\
    where(
      (aircrafts_data.c.range > 3000 ) | 
      (aircrafts_data.c.range < 9000)
    ) 
print(sql_query, '\n')

sql_query = select([aircrafts_data]).\
    where(    
      ~(aircrafts_data.c.aircraft_code == '733')
    ) 
print(sql_query, '\n')

sql_query = select([aircrafts_data]).\
    where(
      ~(aircrafts_data.c.aircraft_code == '733') &
      (aircrafts_data.c.range < 3000)
    )
print(sql_query)

IS NULL

In [ ]:
sql_query = select(flights).\
    where(
      (flights.c.actual_departure == None)
    ) 

print(sql_query, '\n')

In [ ]:
result = engine.execute(sql_query);

result.fetchone()

IS NOT NULL

In [ ]:
sql_query = select(flights).where(
flights.c.actual_departure != None)

print(sql_query, '\n')

IN

In [ ]:
sql_query = select(flights).\
    where(flights.c.departure_airport.in_(['DME','VKO']) )

print(sql_query, '\n')

In [ ]:
engine.execute(sql_query).fetchone()

NOT IN

In [ ]:
sql_query = select(flights).\
    where(flights.c.departure_airport.notin_(['DME','VKO']) )

print(sql_query, '\n')

In [ ]:
engine.execute(sql_query).fetchone()

BETWEEN

In [ ]:
sql_query = select(aircrafts_data).\
            where(aircrafts_data.c.range.between(5000,10000) )

print(sql_query, '\n')

In [ ]:
engine.execute(sql_query).fetchall()

LIKE

In [ ]:
sql_query = select(aircrafts_data).\
            where(aircrafts_data.c.aircraft_code.like('C%') )

print(sql_query, '\n')

In [ ]:
engine.execute(sql_query).fetchall()

Сортировка ORDER BY

In [ ]:
sql_query = select(aircrafts_data).\
            order_by(aircrafts_data.c.range)

print(sql_query, '\n')

In [ ]:
engine.execute(sql_query).fetchall()

Обратная сортировка

In [ ]:
from sqlalchemy import desc

In [ ]:
sql_query = select(aircrafts_data).\
            order_by(desc(aircrafts_data.c.range))

print(sql_query, '\n')

In [ ]:
engine.execute(sql_query).fetchall()

LIMIT

In [ ]:
from sqlalchemy import desc

sql_query = select(aircrafts_data).\
            order_by(desc(aircrafts_data.c.range)).\
            limit(3)

print(sql_query, '\n')

In [ ]:
engine.execute(sql_query).fetchall()

OFFSET

In [ ]:
from sqlalchemy import desc

sql_query = select(aircrafts_data).\
            order_by(desc(aircrafts_data.c.range)).\
            limit(3).\
            offset(3)

print(sql_query, '\n')

In [ ]:
engine.execute(sql_query).fetchall()

LABEL (ALIAS)

In [ ]:
from sqlalchemy import desc

sql_query = select([
                aircrafts_data.c.aircraft_code
               ,aircrafts_data.c.range.label('rng')
            ]).\
            order_by(desc('rng')).\
            limit(3)

print(sql_query, '\n')

In [ ]:
engine.execute(sql_query).fetchall()

Использование функций

In [ ]:
from sqlalchemy.sql import func

functions = [
    
    ## datetime operations
    
    func.timeofday(),
    func.localtime(),
    func.current_timestamp(),    
    func.date_part("month", func.now()),        
    func.now(),
    
    # math operations
    
    func.pow(4,2),
    func.sqrt(441),
    func.pi(),        
    func.floor(func.pi()),
    func.ceil(func.pi()),
    
    # operations with strings
    
    func.lower("ABC"),
    func.upper("abc"),
    func.length("abc"),
]

select_query = select(functions)
print(select_query)

# result_set = engine.execute(select_query)
# print(result_set.fetchall())

Агрегирующие функции

In [ ]:
# Примеры:

agg_functions = [ 
    func.sum(table_name.c.column_name),
    func.avg(table_name.c.column_name),    
    func.max(table_name.c.column_name),
    func.min(table_name.c.column_name),
    func.count(table_name.c.column_name),    
]

In [ ]:
from sqlalchemy.sql import func

sql_query = select([
                func.sum(ticket_flights.c.amount).label('total_amount')
            ])

print(sql_query)

In [ ]:
engine.execute(sql_query).fetchone()

Группировка GROUP BY

In [ ]:
sql_query = select(
                flights.c.departure_airport.label('dep_air')
               ,func.count("*")
            ).group_by('dep_air')

In [ ]:
engine.execute(sql_query).fetchall()

**Объединения JOINS**

In [ ]:
print(aircrafts_data.join(seats))

In [ ]:
sql_query = select([
    aircrafts_data
    ,seats.c.seat_no
    ,seats.c.fare_conditions
]
).\
select_from(aircrafts_data.\
            join(seats)
           )

In [ ]:
print(sql_query)

In [ ]:
engine.execute(sql_query).fetchall()

**Оконные функции**

In [ ]:
sql_query = select(aircrafts_data.c.aircraft_code
                  ,func.sum(aircrafts_data.c.range).over(
                  partition_by = aircrafts_data.c.aircraft_code,
                  order_by = aircrafts_data.c.aircraft_code
                  )
                  )


print(sql_query)

In [ ]:
engine.execute(sql_query).fetchall()

**Загрузка данных с помощью Pandas**

*Импортируем библиотеки*

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

**Выборка данных с помощью Pandas**

In [ ]:
def replace_procent(sql_query):
    sql_query = sql_query.replace('%', '%%')
    return sql_query

In [ ]:
sql = '''
SELECT t.passenger_name
      ,f.scheduled_departure 
      ,sum(tf.amount) over(PARTITION BY t.passenger_name ORDER BY f.scheduled_departure) AS current_sum
      ,tf.amount
  FROM tickets t 
  JOIN ticket_flights AS tf USING (ticket_no) 
  JOIN flights AS f ON tf.flight_id = f.flight_id
 WHERE t.passenger_name LIKE '%OVA'
'''

In [ ]:
sql

In [ ]:
new_sql = replace_procent(sql)

In [ ]:
new_sql

In [ ]:
?pd.read_sql

In [ ]:
df = pd.read_sql(new_sql, con=engine)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df[df['passenger_name'] == 'ADELINA POPOVA'][['scheduled_departure', 'current_sum']]

Посчитаем количество перелётов для каждого месяца

In [ ]:
sql = '''
SELECT extract(YEAR FROM scheduled_departure) AS "year"
      ,extract(MONTH FROM scheduled_departure) AS "month"
      ,count(flight_id) AS count_of_flights
  FROM flights
 GROUP BY extract(YEAR FROM scheduled_departure), extract(MONTH FROM scheduled_departure)
 ORDER BY 1, 2
'''

In [ ]:
df = pd.read_sql(sql, con=engine)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.dtypes

Преобразуем типы двух колонок

In [ ]:
df.year = df.year.astype(int)
df.month = df.month.astype(int)

In [ ]:
df.head()

Создадим новую колонку год-месяц

In [ ]:
(df.year.astype(str) + '-' + df.month.astype(str))

In [ ]:
df['year_month'] = (df.year.astype(str) + '-' + df.month.astype(str))

In [ ]:
df.head()

Нарисуем столбчатую диаграмму кол-ва перелётов по месяцам

In [ ]:
df = df[['year_month', 'count_of_flights']]
ax = df.plot.bar(x='year_month', y='count_of_flights', title='Count of flights')